#

In [11]:
import cv2
import numpy as np
import os
import time
from datetime import datetime
from google.colab import files

# ПАРАМЕТРЫ
MAX_CORNERS = 500            # макс. число точек для LK
MIN_INLIERS_LK = 15          # мин. количество inliers для гомографии LK
MIN_INLIERS_AKAZE = 8        # мин. количество inliers для гомографии AKAZE
QUALITY_THRESH = 0.01        # порог качества для goodFeaturesToTrack
MIN_DISTANCE = 7             # минимальное расстояние между точками
BLOCK_SIZE = 7               # размер блока для оценки углов

# ИНИЦИАЛИЗАЦИЯ
detector = cv2.AKAZE_create()  # AKAZE для восстановления трекинга
matcher = cv2.BFMatcher(cv2.NORM_HAMMING)  # BFMatcher с Хэммингом

# ФУНКЦИИ
def detect_akaze_features(gray):
    """Детекция AKAZE на сером кадре"""
    return detector.detectAndCompute(gray, None)

def find_homography_akaze(kp1, des1, kp2, des2):
    """Восстановление гомографии через AKAZE и RANSAC"""
    if des1 is None or des2 is None or len(des1) == 0 or len(des2) == 0:
        return None

    try:
        matches = matcher.knnMatch(des1, des2, k=2)
        good_matches = [m for m, n in matches if m.distance < 0.7 * n.distance]
    except:
        matches = matcher.match(des1, des2)
        good_matches = matches if len(matches) >= MIN_INLIERS_AKAZE else []

    if len(good_matches) < MIN_INLIERS_AKAZE:
        return None

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H

def reseed_points_in_roi(gray, corners):
    """Повторная инициализация точек внутри ROI"""
    if corners is None or len(corners) < 3:
        return None

    mask = np.zeros_like(gray)
    cv2.fillPoly(mask, [corners.reshape(-1, 2).astype(np.int32)], 255)

    feature_params = dict(maxCorners=MAX_CORNERS, qualityLevel=QUALITY_THRESH,
                         minDistance=MIN_DISTANCE, blockSize=BLOCK_SIZE)
    return cv2.goodFeaturesToTrack(gray, mask=mask, **feature_params)

def calculate_lk_homography(prev_gray, curr_gray, prev_pts, corners):
    """Расчёт оптического потока LK и гомографии"""
    if prev_pts is None or len(prev_pts) < 8:
        return None, None, False

    lk_params = dict(winSize=(21, 21), maxLevel=3,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

    next_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None, **lk_params)
    if next_pts is None or status is None:
        return None, None, False

    status = status.ravel()
    good_old = prev_pts[status == 1]
    good_new = next_pts[status == 1]

    if len(good_old) < 8:
        return None, None, False

    H, mask = cv2.findHomography(good_old.reshape(-1, 1, 2), good_new.reshape(-1, 1, 2), cv2.RANSAC, 3.0)
    if H is None or mask is None:
        return None, None, False

    inliers = np.sum(mask)
    if inliers >= MIN_INLIERS_LK:
        new_corners = cv2.perspectiveTransform(corners, H)
        mask_flat = mask.ravel() == 1
        new_pts = good_new[mask_flat].reshape(-1, 1, 2) if np.sum(mask_flat) > 0 else good_new
        return new_pts, new_corners, True

    return None, None, False

def visualize_frame(frame, corners, is_lost=False, frame_num=0, points_count=0):
    """Отрисовка рамки, статуса трекинга и информации по кадру"""
    vis = frame.copy()
    if corners is not None:
        box_color = (0, 255, 0) if not is_lost else (0, 0, 255)
        cv2.polylines(vis, [corners.reshape(-1, 2).astype(np.int32)], True, box_color, 3)

    status_text = "ТРЕКИНГ" if not is_lost else "ПОТЕРЯНО"
    status_color = (0, 255, 0) if not is_lost else (0, 0, 255)

    cv2.putText(vis, f"Статус: {status_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)
    cv2.putText(vis, f"Кадр: {frame_num}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.putText(vis, f"Точек: {points_count}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return vis

def track_video(input_path, output_dir="results", show_preview=True, preview_interval=30):
    """Основная функция трекинга"""
    print(f"Обработка: {input_path}")
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Ошибка открытия видео")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    video_name = os.path.splitext(os.path.basename(input_path))[0]
    output_path = os.path.join(output_dir, f"tracked_{video_name}_{timestamp}.mp4")

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # ПЕРВЫЙ КАДР
    ret, first_frame = cap.read()
    if not ret:
        cap.release()
        return

    first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

    # Детекция AKAZE для возможного восстановления трекинга
    init_kp, init_des = detect_akaze_features(first_gray)
    if init_kp is None or len(init_kp) < 10:
        print("Недостаточно точек AKAZE")
        cap.release()
        return

    # Инициализация углов объекта (весь кадр)
    corners = np.float32([[0, 0], [width - 1, 0],
                         [width - 1, height - 1], [0, height - 1]]).reshape(-1, 1, 2)
    initial_corners = corners.copy()

    # Первичные точки для LK
    feature_params = dict(maxCorners=MAX_CORNERS, qualityLevel=QUALITY_THRESH,
                         minDistance=MIN_DISTANCE, blockSize=BLOCK_SIZE)
    prev_pts = cv2.goodFeaturesToTrack(first_gray, mask=None, **feature_params)
    if prev_pts is None or len(prev_pts) < 8:
        print("Недостаточно точек LK")
        cap.release()
        return

    prev_gray = first_gray
    is_lost = False

    vis_first = visualize_frame(first_frame, corners, False, 1, len(prev_pts))
    out.write(vis_first)

    # ЦИКЛ ОБРАБОТКИ КАДРОВ
    total_processed = 1
    tracked_frames = 1
    lost_frames = 0
    recovery_attempts = 0
    successful_recoveries = 0
    start_time = time.time()
    frame_num = 1

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        total_processed += 1
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if is_lost:
            # ВОССТАНОВЛЕНИЕ
            lost_frames += 1
            recovery_attempts += 1

            curr_kp, curr_des = detect_akaze_features(frame_gray)
            H_recovery = find_homography_akaze(init_kp, init_des, curr_kp, curr_des)

            if H_recovery is not None:
                corners = cv2.perspectiveTransform(initial_corners, H_recovery)
                prev_pts = reseed_points_in_roi(frame_gray, corners)
                if prev_pts is not None and len(prev_pts) >= 8:
                    prev_gray = frame_gray
                    is_lost = False
                    successful_recoveries += 1
                    tracked_frames += 1
        else:
            # ОТСЛЕЖИВАНИЕ LK
            new_pts, new_corners, success = calculate_lk_homography(
                prev_gray, frame_gray, prev_pts, corners
            )
            if success:
                prev_pts = new_pts
                corners = new_corners
                prev_gray = frame_gray
                tracked_frames += 1
                is_lost = False
            else:
                is_lost = True

        # ВИЗУАЛИЗАЦИЯ
        pts_count = len(prev_pts) if prev_pts is not None else 0
        vis_frame = visualize_frame(frame, corners, is_lost, frame_num, pts_count)
        out.write(vis_frame)

        frame_num += 1
        if frame_num % 50 == 0:
            elapsed = time.time() - start_time
            fps_current = frame_num / elapsed if elapsed > 0 else 0
            print(f"Прогресс: {frame_num} кадров | FPS: {fps_current:.1f} | Точек: {pts_count}")

    # СТАТИСТИКА
    cap.release()
    out.release()
    total_time = time.time() - start_time

    print("\n" + "="*50)
    print(f"Всего кадров: {total_processed}")
    print(f"Отслежено: {tracked_frames}")
    print(f"Потеряно: {lost_frames}")
    print(f"Попыток восстановления: {recovery_attempts}")
    print(f"Успешных восстановлений: {successful_recoveries}")

    if total_processed > 0:
        success_rate = (tracked_frames / total_processed) * 100
        recovery_rate = (successful_recoveries / max(1, recovery_attempts)) * 100
        print(f"Успешность трекинга: {success_rate:.1f}%")
        print(f"Эффективность восстановления: {recovery_rate:.1f}%")
        print(f"Общее время: {total_time:.2f} сек")
        print(f"Средний FPS: {total_processed/total_time:.1f}")

    print("="*50)
    print(f"Сохранено: {output_path}")

    return output_path

# ЗАГРУЗКА ВИДЕО
print("Трекинг объекта (LK + Гомография)")

try:
    print("Загрузите видеофайл...")
    uploaded = files.upload()
    if not uploaded:
        print("Файл не загружен")
    video_path = list(uploaded.keys())[0]
    print(f"Загружено: {video_path}")

    track_video(video_path, show_preview=False)

except ImportError:
    video_path = input("Введите путь к видео: ").strip()
    if not os.path.exists(video_path):
        print("Файл не найден")


Трекинг объекта (LK + Гомография)
Загрузите видеофайл...


Saving mona-lisa-blur.avi to mona-lisa-blur (4).avi
Загружено: mona-lisa-blur (4).avi
Обработка: mona-lisa-blur (4).avi
Прогресс: 50 кадров | FPS: 53.6 | Точек: 347
Прогресс: 100 кадров | FPS: 71.9 | Точек: 346
Прогресс: 150 кадров | FPS: 82.5 | Точек: 346
Прогресс: 200 кадров | FPS: 88.2 | Точек: 346
Прогресс: 250 кадров | FPS: 93.6 | Точек: 346
Прогресс: 300 кадров | FPS: 96.7 | Точек: 346
Прогресс: 350 кадров | FPS: 81.4 | Точек: 346
Прогресс: 400 кадров | FPS: 77.5 | Точек: 342
Прогресс: 450 кадров | FPS: 77.4 | Точек: 338
Прогресс: 500 кадров | FPS: 79.8 | Точек: 306
Прогресс: 550 кадров | FPS: 82.4 | Точек: 303

Всего кадров: 558
Отслежено: 539
Потеряно: 19
Попыток восстановления: 19
Успешных восстановлений: 1
Успешность трекинга: 96.6%
Эффективность восстановления: 5.3%
Общее время: 6.74 сек
Средний FPS: 82.8
Сохранено: results/tracked_mona-lisa-blur (4)_20251218_211846.mp4
